# Semiautomatic Quality Control / Alignment helper

This notebook is designed to help alignment of an Imspector-driven microscope.

The process is the following:

1. Image fluorescent beads
2. Call cell to fit Gaussian PSFs to all beads
3. Repeat. Results over time are plotted

## Required libraries
** run once **

In [ ]:
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from itertools import count
import numpy as np
import time

from calmutils.localization import refine_point_lsq, detect_dog
from calmutils.localization.util import sig_to_full_width_at_quantile, get_ellipse_params

from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse

import pandas as pd

from specpy import *

%matplotlib notebook
plt.rcParams['figure.figsize'] = [8,8]

## Constants
** run once **

In [ ]:

# axes in Imspecctors preferred order
AXES = ['z', 'y', 'x']
# how many pixels to cut for Gauss fit on each side (times fwhm)
CUT_FOLD_FWHM = 2
# how much the fit fwhm may deviate from 
FIT_CUTOFF = 2


## Detection parameters
* expected FWHM
* intensity threshold for detection

** run once ** (or if expected size/intensity changes drastically)

In [ ]:
# detection threshold
thresh = 0.3
# estimated FWHM (in m!)
# NB: z, y, x - order
fwhm_estim = np.array([400, 40, 40]) * 1e-9

# Actual QC pipeline
## 1. Initialize
** run once ** (or a second time to reset the pipeline)

In [ ]:
# init new dataframe for results
df = pd.DataFrame()

run_counter = count()
imgs_accu = []


## 2. FWHM estimation

Runnung this will get the **currently selected images** in Imspector and perform a Gaussian fit to the beads. It will produce 2 plots for each channel:

* **Image with FWHM ellipses at each detected spot** (for 3d images, this is a z-maximum-intensity-projection)
* **Plot of FWHMS over time** the median FWHM in each dimension in this and all previous runs

Before running this cell again, you should **acquire a new image in Imspector and have it selected**.

In [ ]:
# connect to Imspector, get current images, drop singleton dimensions
im = Imspector()
ms = im.active_measurement()
stack_names = ms.active_configuration().stack_names()

imgs = []
for stack_name in stack_names:
    stack = ms.stack(stack_name)
    img = stack.data().copy()
    img = img.squeeze()
    img = img.astype(np.float)
    imgs.append(img)

# list of all images over time -> save?
imgs_accu.append(imgs)

# custom parameter dict
# use this to add custom information to the resulting table
# e.g. 'correctioncollar' : 0.175
custom_parameters = {}

# get index of this run and stratime of analysis
starttime = pd.to_datetime(time.asctime())
run_idx = next(run_counter)
run_parameters = {
    'starttime' : starttime,
    'run_idx' : run_idx
}

# do analysis for each channel
for (channel_idx, img) in enumerate(imgs):
    
    # plot (maximum projection if 3d)
    plt.figure('Channel {} localizations'.format(channel_idx))
    is3d = len(img.shape)>2
    projection = np.apply_along_axis(np.max, 0, img) if is3d else img
    plt.imshow(projection, interpolation='nearest')

    # get pixelsize, offset, fov
    psz = [ms.active_configuration().parameters('ExpControl/scan/range/{}/psz'.format(a)) for a in AXES[(0 if is3d else 1):]]
    off = [ms.active_configuration().parameters('ExpControl/scan/range/{}/off'.format(a)) for a in AXES[(0 if is3d else 1):]]
    fov = [ms.active_configuration().parameters('ExpControl/scan/range/{}/len'.format(a)) for a in AXES[(0 if is3d else 1):]]
    
    # collect for df
    scan_param_dict = {'channel_index' : channel_idx}    
    scan_param_dict.update({'pixelsize_{}'.format(AXES[i + (0 if is3d else 1)]) : float(psz[i]) for i in range(len(psz)) })
    scan_param_dict.update({'fov_{}'.format(AXES[i + (0 if is3d else 1)]) : float(fov[i]) for i in range(len(psz)) })
    scan_param_dict.update({'offset_{}'.format(AXES[i + (0 if is3d else 1)]) : float(off[i]) for i in range(len(psz)) })
    
    # get initial guess
    guess = detect_dog(img, threshold=thresh, fwhm=fwhm_estim[(0 if is3d else 1):], pixsize=psz)
    for g in guess:
        
        # cut region size
        cut = list(np.ceil(fwhm_estim[(0 if is3d else 1):] / np.array(psz) * CUT_FOLD_FWHM).astype(int))
        
        # do Gauss fit, ignore if we cannot fit
        refined, p = refine_point_lsq(img, g, cut)
        if p is None:
            continue
          
        # ignore other part of fit result
        p, _ = p
        
        fit_param_dict = {}
        fit_param_dict.update(dict(zip(
            ['background', 'peak_height'],
            [p[0], p[1]]
        )))
        fit_param_dict.update({'fit_mu_{}'.format(AXES[i + (0 if is3d else 1)]) : list(refined)[i] * psz[i] for i in range(len(img.shape)) })
        fwhm_fit = sig_to_full_width_at_quantile(p[2+len(img.shape):]) * np.array(psz)
        fit_param_dict.update({'fit_fwhm_{}'.format(AXES[i + (0 if is3d else 1)]) : fwhm_fit[i] for i in range(len(img.shape)) })
        
        # get mu and cov for ellipse plotting
        mu = list(refined)[(1 if is3d else 0):] # we drop z if 3d
        cov = p[2+len(img.shape):]
        
        
        # fitted FWHM deviates too much from expectation -> ignore
        if np.any(sig_to_full_width_at_quantile(cov) > fwhm_estim[(0 if is3d else 1):] / np.array(psz) * FIT_CUTOFF):
            continue
        
        cov = cov[(1 if is3d else 0):]
        cov = np.diag(cov)
    
        e = Ellipse(list(reversed(mu)), *get_ellipse_params(cov**2), fill=None, color='red', linewidth=2)
        plt.gca().add_artist(e)
        
        # append results for this spot to table
        row = {}
        row.update(custom_parameters)
        row.update(run_parameters)
        row.update(scan_param_dict)
        row.update(fit_param_dict)
        df = df.append(row, ignore_index=True)
        
    plt.show()
    
# plot simple summary (timeseries of sigmas)
for channel, group_df in df.groupby(['channel_index']):
    
    plt.figure('Channel {} FWHMS'.format(channel))
    
    xfwhms = [ (run, group_df2.fit_fwhm_x.median()) for (run, group_df2) in group_df.groupby(['run_idx']) ]
    xfwhms.sort(key=lambda x: x[0]) # sort by run_id
    plt.plot([x[0] for x in xfwhms], [x[1] for x in xfwhms], '*-', label='FWHM (x)')
    
    yfwhms = [ (run, group_df2.fit_fwhm_y.median()) for (run, group_df2) in group_df.groupby(['run_idx']) ]
    yfwhms.sort(key=lambda x: x[0]) # sort by run_id
    plt.plot([x[0] for x in yfwhms], [x[1] for x in yfwhms], '*-', label='FWHM (y)')
    
    # only plot z if we have values for it
    if 'fit_fwhm_z' in group_df.columns:
        zfwhms = [ (run, group_df2.fit_fwhm_z.median()) for (run, group_df2) in group_df.groupby(['run_idx']) ]
        zfwhms.sort(key=lambda x: x[0]) # sort by run_id
        plt.plot([x[0] for x in zfwhms], [x[1] for x in zfwhms], '*-', label='FWHM (z)')
    
    plt.legend()
    plt.show()

## 3. save results

The localizations in each run are saved to a table under the hood. If you want to keep them, specify a save path (.csv file) and run this cell to save the table.

In [ ]:
savepath = 'C:/Users/RESOLFT/Desktop/localizations_001.csv'

# save table
df.to_csv(savepath, index=False)

# TODO: save images and all parameters as well? HDF5?
    